In [ ]:
pip install selenium # Installing selenium package that is used to automate web browser interaction from Python.


In [ ]:
pip install awscli  # Installing awscli package provides a unified command line interface to Amazon Web Services.

In [ ]:
pip install boto3  # This allows Python developers to write software that makes use of services like Amazon S3 and Amazon EC2.

In [ ]:
pip install webdriver-manager #

In [ ]:
# Loading modules
#!pip install webdriver-manager
import awscli
import boto3
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

####SCRAPE THE WEBSITE######
### Call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

# Entering  the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

# Recognize xpath of location to select element
inputElement = browser.find_element(By.XPATH,"/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element(By.XPATH,"/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()
sleep(4) # Adding a sleep element that will allow for the page to load
# Identifying the table to scrape
table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
sleep(1)
#####CREATING DATE FRAME#####
# Creating empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements(By.CSS_SELECTOR,'tr'):
      cols = df.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3



In [ ]:
import boto3
from botocore.exceptions import ClientError

# Assigning the bucket name against the bucket variable
bucket_name = 'm10-assignments'

aws_access = input('Enter your access key')
aws_secret = input('Enter your secret key')

aws_s3_client = boto3.client('s3',
                aws_access_key_id = aws_access,
                aws_secret_access_key = aws_secret)

# Below commented code has the access key to my AWS account. 
response = aws_s3_client.list_buckets()
bucket_exist = False

for bucket in response['Buckets']:
    if bucket['Name'] == bucket_name:
        bucket_exist = True
        break

if bucket_exist:
    print("The bucket exists")
else:
    print("The bucket doesn't exist")

# Create the bucket if it doesn't exist with the help of try, except block
if not bucket_exist:
    try:
        aws_s3_client.create_bucket(Bucket=bucket_name)
        print(f"{bucket_name} bucket has been created on my AWS S3")
    except ClientError as e:
        print(e)
        print(f"{bucket_name} bucket cannot be created on S3")
    except:
        print(f"{bucket_name} cannot be created on S3")

In [ ]:
from io import StringIO
def upload_s3(df,i):
    global aws_s3_client, bucket_name
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, header=True, line_terminator='\n')
    csv_buffer.seek(0)
    aws_s3_client.put_object(Bucket=bucket_name, Body=csv_buffer.getvalue(), Key=i)

# Checking if the bucket exists

response = aws_s3_client.list_buckets()
bucket1  = [bucket['Name'] for bucket in response['bucket1']]
if bucket_name not in bucket1:
    print(f"{bucket_name} bucket does not exist.")
else:
    print("Uploading Data")
    upload_s3(df, filenames3)
    print("Data uploaded successfully ")

filename= 'charities_bureau_scrape_group1'# Name of the group
datetime = time.strftime("%Y%m%d%H%M%S") # Timestamp
filenames3 = "%s%s.csv"%(filename,datetime) # Name of the filepath in S3 and csv file

#print the success message
print("Successfully uploaded file to location:"+str(filenames3))